### Exercise based on [this article](https://thelaziestprogrammer.com/sharrington/math-of-machine-learning/solving-logreg-newtons-method)

### Load data from UCI database

Data source = south boston housing

In [1]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'

cols = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'bk', 'lstat', 'medv']

df = pd.read_csv(url, header = None, names = cols, delim_whitespace = True)

In [2]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,bk,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


### Create input vector & target vector

- input feature vector = `medv`, the median of historical value each house in $1k
- target vector = `rm`, the number of rooms per house. further turned into binary to indicate if `rm` >= 6

In [3]:
X = df.medv.values
X = (X - X.mean()) / (X.max() - X.min())
X = np.hstack((X.reshape(-1, 1), np.ones(len(X)).reshape(-1, 1)))

In [4]:
y = np.where(df.rm.values >= 6, 1, 0)

### Code for algorithm

In [5]:
# create class 

class nt_logistic_regression():
    
    # initialize model & assign parameters
    def __init__(self, n = 1000, tolerance = 0.000001):
        
        self.thetas = np.random.normal(size = 2)
        self.n_iter = n
        self.tol = tolerance
     
    # define sigmoid function
    def __sigmoid(self, X, thetas):
        
        z = X.dot(thetas)
        
        return 1. / (1. + np.exp(-z))
    
    # define log likelihood function
    def __log_likelihood(self, X, y, thetas):
        
        sig_prob = self.__sigmoid(X, thetas)
        
        return np.sum(y * np.log(sig_prob) + (1 - y) * np.log(1 - sig_prob))
    
    # define gradient function
    def __gradient(self, X, y, thetas):
        
        sig_prob = self.__sigmoid(X, thetas)
        
        # returns a n=2 vector that contains gradient of the log likelihood function
        return np.array([[np.sum((y - sig_prob) * X[:, 0]), np.sum((y - sig_prob) * 1)]])
    
    # define hessian function (hard code, not dynamic)
    def __hessian(self, X, y, thetas):
        
        sig_prob = self.__sigmoid(X, thetas)
        
        d1 = np.sum((sig_prob * (1 - sig_prob)) * X[:, 0] * X[:, 0])
        d2 = np.sum((sig_prob * (1 - sig_prob)) * X[:, 0] * 1)
        d3 = np.sum((sig_prob * (1 - sig_prob)) * 1 * 1)
        
        H = np.array([[d1, d2], [d2, d3]])
        
        return H
    
    
    # define train function to bring everything together
    def train(self, X, y):
        
        L = self.__log_likelihood(X, y, self.thetas)
        
        L_dt = np.Inf
        
        cnt = 0
        
        while L_dt > self.tol or cnt <= self.n_iter:
            
            cnt += 1
            
            g = self.__gradient(X, y, self.thetas)
            H_inv = np.linalg.inv(self.__hessian(X, y, self.thetas))
            
            theta_dt = H_inv.dot(g.T).ravel()
            
            self.thetas += theta_dt
            
            L_new = self.__log_likelihood(X, y, self.thetas)
            L_dt = L - L_new
            L = L_new
    
        return self.thetas
        
        

In [6]:
test = nt_logistic_regression()
test.train(X, y)

array([6.61968184, 0.92577422])

In [7]:
# check against sklearn 

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C= 1000000, fit_intercept=False)
lr.fit(X, y)

/Users/RayneYunZhang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1000000, class_weight=None, dual=False,
          fit_intercept=False, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [8]:
lr.coef_

array([[6.61935597, 0.92573953]])